# Stemningsanalyse i Facebookgrupper

Denne notesbog kan hjælpe med at holde styr på stemmingen i Facebookgrupper. [AFINN](https://github.com/fnielsen/afinn) bruges til sentimentanalyse af beskeder, kommentarer og vedhæftninger i en gruppe. Herved er det muligt at veje stemningen i gruppen over tid og fordelt på forskellige kategorier som f.eks. gadelygter og affald. Klassifikationen af kategorier sker ved at søge efter beskeder som indeholder udvalgte ord som har relation til kategorien.

## Indhold
1. [Input](#Input)
2. [Tilretning](#Tilretning)
3. [Berigelse](#Berigelse)
4. [Analyse](#Analyse)
5. [Output](#Output)

In [ ]:
import numpy as np
import pandas as pd
import requests
from afinn import Afinn
from IPython.display import HTML
import psycopg2
from sqlalchemy import create_engine

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 100)
#pd.reset_option('display.max_colwidth')

# Input 
## Hent data og lav dataframe

In [ ]:
# Angiv de grupper der skal hentes feeds fra ( Slå gruppe id op http://lookup-id.com/ )
groups = {
    "Mit Ballerup": '241841222854414', # Denne gruppe er nu lukket
    "Vores by Skovlunde": '725746707539505',
    "Mit Måløv": '272828512895426'
}
access_token = 'xxx|yyy'
limit = 500 # facebook har max limit på 500 
feeds = []

for group, group_id in groups.items(): 
    url = "https://graph.facebook.com/v2.10/{}/feed?fields=reactions,comments,message, created_time,attachments&limit={}&access_token={}".format(group_id, limit, access_token)
    try:
        r = requests.get(url)
        data = r.json()
        df_data = data['data']
        df = pd.DataFrame(df_data)
        df['kilde'] = group
        feeds.append(df)
        
    except Exception as e:
        print('Exception:',e)

In [ ]:
# Samler feeds fra grupperne i en DF
df = pd.concat(feeds, ignore_index=True)

# Tilretning
Tilretter DF med udfra de eksisterende oplysninger

In [ ]:
#dato laves til datatype datetime
df['created_time'] = pd.to_datetime(df['created_time'])

In [ ]:
df['link'] = df['id'].apply(lambda x: 'https://www.facebook.com/' + x)
#df['link'] = df['id'].apply(lambda x: '<a href="https://www.facebook.com/{0}" target="_blank">link</a>'.format(x))

In [ ]:
# Tittel og beskrivelse fra vedhæftelser som f.eks. billeder tilføjes
def att_text(series):
    try:
        return series['data'][0]['title'] + ' - ' + series['data'][0]['description']
    except:
        pass
     
df['attachments'] = df['attachments'].apply(att_text)

In [ ]:
def comment_list(col):
    if pd.notnull(col):
        return [ x['message'] for x in col['data']]
    else:
        None

df['kommentarer'] = df['comments'].apply(comment_list)

#### Reaktioner på FB

In [ ]:
def reaction(col, mood):
    if pd.notnull(col):
        return [ x['type'] for x in col['data']].count(mood)
    else:
        None

df['likes'] = df['reactions'].apply(lambda x: reaction(x, 'LIKE'))
df['angry'] = df['reactions'].apply(lambda x: reaction(x, 'ANGRY'))
df['love'] = df['reactions'].apply(lambda x: reaction(x, 'LOVE'))
df['total_reactions'] =  df['reactions'].apply(lambda x: len(x['data']) if pd.notnull(x) else None)

# Berigelse
Beriger DF med stemning og klassifikation

## Stemning
AFINN bruges til at bestemme stemningen i besked og kommentar

In [ ]:
afinn = Afinn(language='da', emoticons=True)

In [ ]:
# Beskedstemning
df['beskedstemning'] = df['message'].apply(lambda x: afinn.score(str(x)))

In [ ]:
# Kommentarstemning
def comment_rating(series):
    if pd.notnull(series):
        return sum([ afinn.score(x['message']) for x in series['data'] ])
    else:
        None

df['kommentarstemning'] = df['comments'].apply(comment_rating)

In [ ]:
# Attachment stemning
df['attachmentstemning'] = df['attachments'].apply(lambda x: afinn.score(str(x)))

## Klassifikation
Beskeder klassificeres afhængigt af kategori

In [ ]:
keywords = {
    'belysning': 'lys lyset lyser belysning belysningen lampe lamper lamperne gadelampe gadelamper gadelamperne lygte lygter lygterne gadelygte gadelygter gadelygterne gadebelysning mørkt mørke lyskryds lyskrydset lyssignal'.split(' '),
    'skrald': 'skrald affald skraldespand renovation affaldscontainer container affaldsspand affaldsbeholder beholder skraldebeholder affaldskube skraldesortering'.split(' '),
    'valg': 'kommunevalg, kommunevalget, stemmeseddel, stemmesedlen, stemmesedler, kv17'.split(' '),
}

In [ ]:
def classify(msg, keywords):
    '''
    Klassificerer tekst udfra kategorier med tilhørende nøgleord. 
    Er afhængig af AFINN split til at konvertere besked til liste med ord
    
    Parameters
    ----------
    msg:
        besked som ønskes klassificeres
    keywords:
        dictionaty med kategori som key liste med nøgleord som value 
    '''
    
    categories = []
    
    if pd.notnull(msg):
        for cat, words in keywords.items():
            if any(word.lower() in afinn.split(msg.lower()) for word in words):
                categories.append( cat )
            else:
                None
    else:
        None
        
    return categories

In [ ]:
df['kategori'] = df['message'].apply(lambda x: ', '.join(classify(x, keywords)))

## Endelig dataframe

In [ ]:
bal = df[['created_time','kilde','message','attachments','beskedstemning','kommentarstemning', 'attachmentstemning', 'total_reactions', 'likes','angry','love', 'kategori', 'link']]
#HTML(bal.to_html (escape=False))

In [ ]:
bal.info()

In [ ]:
bal[bal['kategori'] == 'belysning'].shape

# Analyse

In [ ]:
bal = df[['created_time','kilde','message','attachments','beskedstemning','kommentarstemning', 'attachmentstemning', 'total_reactions', 'likes','angry','love', 'kategori', 'link']]
HTML(bal.to_html (escape=False))

## Samlet stemning i de sidste 500 beskeder om belysning på Facebook
Man kunne evt kigge på at lave et stemningsindex:<br>
```stemningsindex = (stemning + kommentarstemning) * (likes + angry)```

In [ ]:
bal['kommentarstemning'].sum() + bal['beskedstemning'].sum()

In [ ]:
bal.groupby('kilde')['beskedstemning'].describe()

## Visualisering

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
belysning

In [ ]:
#sns.tsplot(belysning, time='created_time', value='beskedstemning', condition='kilde')

test = belysning.set_index('created_time')['beskedstemning']
sns.tsplot(test)

In [ ]:
sns.distplot(test)


In [ ]:
plt.figure(figsize=(10, 7))
sns.barplot(x='kilde', y='beskedstemning', data=belysning.groupby('kilde').sum().reset_index())

# Output
Data sendes til PostgreSQL

In [ ]:
user = 'xx'
pw = 'xx'
port = 5432
host = 'xx'
db = 'xx'
schema = 'xx'
table = 'xx''

In [ ]:
con = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, pw, host, port, db))

In [ ]:
bal.to_sql(table, con, schema=schema, if_exists='replace', index_label='gid')

In [ ]:
import sqlalchemy

In [ ]:
dtype = {
    'attachments': sqlalchemy.JSON(), 
    'comments': sqlalchemy.JSON(), 
    'created_time': sqlalchemy.TIMESTAMP(), 
    'id': sqlalchemy.TEXT(), 
    'message': sqlalchemy.TEXT(), 
    'reactions': sqlalchemy.JSON(),
    'kilde': sqlalchemy.VARCHAR()
}

In [ ]:
df.to_sql('ballerup_raw', con, schema=schema, if_exists='replace', index_label='gid', dtype=dtype)

# Sandkasse

In [ ]:
afinn.split('Hej med Dig!!!'.lower())

In [ ]:
import string
import nltk

## Stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
#stopwords.words('danish')

## Stemming

In [ ]:
from nltk.stem.snowball import DanishStemmer

In [ ]:
stemmer = DanishStemmer()

In [ ]:
test = 'Bilerne kørte mod højre i går morges og i morges'
[stemmer.stem(word) for word in afinn.split(test.lower())]

## Frequency Distribution

In [ ]:
from nltk import FreqDist

In [ ]:
freqdist = FreqDist()

In [ ]:
freqdist.freq(test)

## Maskinelæring

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
#nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

def text_process(message):
    '''
    Beskederne bliver renset fra stopord og tegn og fyldt i liste. Afhængig af NLTK Stopwords og AFINN split.
    '''
    if pd.notnull(message):
        splt_msg = afinn.split(message)
        return [word for word in splt_msg if word.lower() not in stopwords.words('danish')]
    else:
        pass

In [ ]:
tags = '''lys lyset lyser belysning belysningen lampe lamper lamperne gadelampe gadelamper gadelamperne 
lygte lygter lygterne gadelygte gadelygter gadelygterne gadebelysning mørkt mørke'''.split(' ')

In [ ]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(df['message'].fillna(''))
vectorizer = CountVectorizer(analyzer=text_process, vocabulary=tags)
words_matrix = vectorizer.fit_transform(df['message'].fillna(''))

test = pd.DataFrame(data=words_matrix.todense(), columns=vectorizer.get_feature_names())

In [ ]:
test

In [ ]:
# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

https://stackoverflow.com/a/40686775/3914186